### Multi-node distributed training of *Hugging Face* models with *PyTorch*, *Horovod*, *Petastorm*, and *DeepSpeed*

##### Here we perform fine tuning of a pre-trained NLP model from [*Hugging Face*](https://huggingface.co/). We use [*PyTorch*](https://pytorch.org/) as the Deep Learning framework for model training and [*Horovod*](https://horovod.ai/) to provide the distributed training infrastructure on *Spark*. [*Petastorm*](https://petastorm.readthedocs.io/en/latest/index.html) is used to integrate the distributed data layer on *Spark* with *PyTorch*. [*DeepSpeed*](https://www.microsoft.com/en-us/research/project/deepspeed/) provides the infrastructure for optimized model training; here we use its implementation of the [*Zero Redundancy Optimizer (ZeRO)*](https://www.deepspeed.ai/tutorials/zero/).

Import the necessary packages.

In [0]:
import os
import time
import random
import pickle
import numpy as np

import torch
from torch.optim import AdamW
import transformers
from transformers import AutoConfig, AutoModelForSequenceClassification, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, SequentialSampler
from torch.utils.data.distributed import DistributedSampler

from petastorm.spark import SparkDatasetConverter, make_spark_converter
from petastorm import TransformSpec

import horovod.torch as hvd
from sparkdl.horovod import log_to_driver
from sparkdl import HorovodRunner

import deepspeed

/databricks/python_shell/dbruntime/PythonPackageImportsInstrumentation/__init__.py:171: FutureWarning: pyarrow.LocalFileSystem is deprecated as of 2.0.0, please use pyarrow.fs.LocalFileSystem instead.
  original_result = python_builtin_import(name, globals, locals, fromlist, level)
/databricks/python/lib/python3.9/site-packages/petastorm/spark/spark_dataset_converter.py:28: FutureWarning: pyarrow.LocalFileSystem is deprecated as of 2.0.0, please use pyarrow.fs.LocalFileSystem instead.
  from pyarrow import LocalFileSystem


Define the number of workers in the cluster and the folders in the Databricks File System to load the prepared data from.

In [0]:
num_workers = 8

train_data_folder = '/train_data'
val_data_folder = '/val_data'

Read the training data into a Spark DataFrame.

In [0]:
sdf_tokens_train = spark.read.parquet(os.path.join(train_data_folder, 'tokens_train_data'))
display(sdf_tokens_train.select(sdf_tokens_train.columns).limit(10))

id,label,input_ids,token_type_ids,attention_mask
17179869184,1,"List(1, 2128, 359, 386, 53935, 12408, 341, 271, 298, 262, 668, 265, 319, 269, 266, 3954, 280, 268, 934, 271, 62970, 4083, 341, 271, 266, 3954, 280, 268, 2673, 280, 268, 11422, 296, 1079, 274, 360, 323, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)","List(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)","List(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)"
17179869185,0,"List(1, 2364, 366, 1966, 366, 8791, 2648, 366, 304, 5660, 1607, 323, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)","List(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)","List(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)"
17179869186,0,"List(1, 262, 1090, 265, 262, 4278, 269, 3211, 264, 2452, 497, 10970, 1945, 5930, 343, 269, 366, 1616, 278, 280, 268, 310, 5550, 354, 21276, 718, 8001, 452, 3788, 323, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)","List(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)","List(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)"
17179869187,1,"List(1, 262, 305, 271, 83426, 2648, 269, 16362, 415, 22455, 13913, 323, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

Read the validation data into a Spark DataFrame.

In [0]:
sdf_tokens_val = spark.read.parquet(os.path.join(val_data_folder, 'tokens_val_data'))
display(sdf_tokens_val.select(sdf_tokens_val.columns).limit(10))

id,label,input_ids,token_type_ids,attention_mask
60129542144,0,"List(1, 1145, 265, 833, 264, 13605, 347, 12191, 268, 366, 647, 5206, 39755, 592, 403, 282, 32035, 579, 266, 1863, 275, 315, 3309, 2026, 322, 267, 266, 1421, 291, 966, 323, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)","List(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)","List(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)"
60129542145,0,"List(1, 266, 11752, 1302, 265, 374, 366, 275, 11561, 263, 5622, 288, 262, 674, 265, 978, 280, 2835, 263, 4278, 9802, 283, 103951, 283, 7128, 4830, 366, 18712, 263, 7548, 4028, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)","List(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)","List(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)"
60129542146,0,"List(1, 759, 5550, 702, 280, 297, 488, 400, 1215, 323, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)","List(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)","List(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)"
60129542147,0,"List(1, 8387, 12116, 1860, 2787, 334, 266, 1234, 933, 293, 375, 1860, 328, 10679, 409, 264, 282, 22133, 77673, 15038, 101782, 335, 306, 1618, 322, 323, 304, 306, 1838, 308, 20772, 280, 268, 843, 263, 306

Here we set a folder in the Databricks File System to be used by *Petastorm* as a cache for intermediate data. We then create the [*Petastorm SparkDatasetConverter*](https://petastorm.readthedocs.io/en/latest/api.html#module-petastorm.spark.spark_dataset_converter) objects that we later use for reading data from *Spark DataFrame* obects into *PyTorch DataLoader* objects.

In [0]:
# Set a cache directory on DBFS FUSE for intermediate data.
spark.conf.set(SparkDatasetConverter.PARENT_CACHE_DIR_URL_CONF, 'file:///dbfs/tmp/petastorm/cache')

converter_train = make_spark_converter(sdf_tokens_train, dtype=None)
converter_val = make_spark_converter(sdf_tokens_val, dtype=None)

/databricks/python/lib/python3.9/site-packages/petastorm/fs_utils.py:88: FutureWarning: pyarrow.localfs is deprecated as of 2.0.0, please use pyarrow.fs.LocalFileSystem instead.
  self._filesystem = pyarrow.localfs
The median size 110375 B (< 50 MB) of the parquet files is too small. Total size: 886452 B. Increase the median file size by calling df.repartition(n) or df.coalesce(n), which might help improve the performance. Parquet files: file:/dbfs/tmp/petastorm/cache/20220914161216-appid-app-20220914152732-0000-2bd47402-4a7d-45c2-8007-1caf51d1b2cd/part-00001-tid-2632364793384534384-eb8abf13-ee88-4c1a-9d36-b6ccba961445-101-1-c000.parquet, ...
The median size 17235 B (< 50 MB) of the parquet files is too small. Total size: 137167 B. Increase the median file size by calling df.repartition(n) or df.coalesce(n), which might help improve the performance. Parquet files: file:/dbfs/tmp/petastorm/cache/20220914161217-appid-app-20220914152732-0000-a0300bd4-4a01-4336-9c2b-b05bd1745fc3/part-00000

Here is where we define the main function of this notebook, which specifies the model fine tuning loop as a [*PyTorch distributed training* process with *Horovod*](https://docs.microsoft.com/en-us/azure/databricks/applications/machine-learning/train-model/distributed-training/horovod-spark), which we [*optimize with DeepSpeed*](https://www.deepspeed.ai/training/) to be run on the worker nodes in our cluster.

In [0]:
def finetune_model():
    
    # initiate Horovod in the worker node
    ######
    hvd.init()
    ######

    # ranks are the unique Horovod process identifiers
    # local_rank refers to the process(es) running on a node
    # rank refers to the processes running on the entire cluster (all participating nodes)
    # world_size is the size of the cluster in number of nodes in a cluster
    ######
    local_rank = hvd.local_rank()
    rank = hvd.rank()
    world_size = hvd.size()
    ######
    
    # rank 0 usually is the node that triggers the MPI job execution in a Horovod cluster
    # when we need to run something only in a single node in the cluster, we usually use the rank 0 node
    ######
    model_folder = '/dbfs/model_outputs'
    if rank == 0:
        os.makedirs(model_folder, exist_ok = True)
        log_to_driver(f'Cluster size: {world_size} GPUs')
    ######
    
    # set the GPU device for PyTorch
    ######
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    if device.type == 'cuda':
        torch.cuda.set_device(hvd.local_rank())
        
    torch.cuda.empty_cache()
    ######
    
    # set random seeds for reproducibility
    ######
    seed_val = 111
    random.seed(seed_val)
    np.random.seed(seed_val)
    torch.manual_seed(seed_val)
    torch.cuda.manual_seed_all(seed_val)
    ######

    # here we instantiate the pre-trained model from Hugging Face
    # then we instantiate the optimizer and scheduler for model training
    ######
    model_type = 'microsoft/deberta-v3-base'
    max_length = 128
    hidden_dropout_prob = 0.2
    attention_probs_dropout_prob = 0.3
    num_labels = 2
    batch_size = 96
    learning_rate = 2e-6 * world_size
    num_epochs = 100

    config = AutoConfig.from_pretrained(model_type, hidden_dropout_prob=hidden_dropout_prob, attention_probs_dropout_prob=attention_probs_dropout_prob, num_labels=num_labels)
    model = AutoModelForSequenceClassification.from_pretrained(model_type, config=config)

    adam_epsilon = 1e-8
    no_decay = ['bias', 'LayerNorm.weight']

    optimizer_grouped_parameters = [
      {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
       'weight_decay_rate': 0.2},
      {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
       'weight_decay_rate': 0.0}
    ]
    
    optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate, eps=adam_epsilon)
    
    total_steps = len(converter_train) * num_epochs
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)
    
    # here we specify the DeepSpeed configuration
    # then we wrap the model, optimizer, and scheduler with DeepSpeed
    # the returned model_engine object will be used for the forward and backward passes during training
    ######
    deepspeed_cfg = {'train_batch_size': batch_size,
                     'gradient_accumulation_steps': 1,
                     'fp16': {'enabled': True},
                     'zero_optimization': {'stage': 2}}
    
    model_engine, _, _, _ = deepspeed.initialize(model=model, optimizer=optimizer, lr_scheduler=scheduler, model_parameters=model.named_parameters(), config=deepspeed_cfg)
    ######
    
    # a simple function to get an elapsed time in minutes and seconds
    ######
    def epoch_time(start_time, end_time):
        elapsed_time = end_time - start_time
        elapsed_mins = int(elapsed_time / 60)
        elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
        return elapsed_mins, elapsed_secs
    ######

    # initiate the variables to be used in the training loop
    ######
    train_losses = []
    val_losses = []

    best_loss = None
    early_stop_min_delta = 0
    early_stop_counter = 0
    early_stop_patience = 5
    flag_tensor = torch.zeros(1).to(device)
    ######
    
    # we create the PyTorch DataLoader objects from the Petastorm SparkDatasetConverter objects we defined earlier
    with converter_train.make_torch_dataloader(cur_shard=rank, shard_count=world_size, batch_size=batch_size) as train_dataloader, converter_val.make_torch_dataloader(cur_shard=rank, shard_count=world_size, batch_size=batch_size) as val_dataloader:
        
        # define the iterators on top of the PyTorch DataLoader objects, to get batch data from them
        ######
        train_dataloader_iter = iter(train_dataloader)
        train_steps = len(converter_train) // (batch_size * world_size)
        
        val_dataloader_iter = iter(val_dataloader)
        val_steps = max(1, len(converter_val) // (batch_size * world_size))
        ######

        # this is the main training loop, where we fine tune the pre-trained model
        ######
        for n in range(num_epochs):
            train_loss = 0
            val_loss = 0
            start_time = time.time()

            # this block runs one epoch over the training data, running forward and backward passes and updating the model weights
            # we use Horovod to compute the average loss for tracking purposes only
            # the actual loss used by the backward pass is the individual loss on each node
            # the distributed optimizer takes care of averaging and synchronizing the model weights across all nodes
            ######
            model_engine.train()
            for step in range(train_steps):
                batch = next(train_dataloader_iter)
                input_ids = batch['input_ids'].to(device)
                token_type_ids = batch['token_type_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['label'].to(device)
                outputs = model_engine(input_ids=input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask, labels=labels)
                loss = outputs.loss
                model_engine.backward(loss)
                model_engine.step()
                train_loss += loss.data / train_steps
                train_loss = hvd.allreduce(train_loss, op=hvd.Sum)
                train_loss = train_loss / world_size
            ######

            # this block runs one epoch over the validation data
            # we use Horovod to average the validation losses for tracking purposes
            ######
            with torch.no_grad():
                model_engine.eval()
                for step in range(val_steps):
                    batch = next(val_dataloader_iter)
                    input_ids = batch['input_ids'].to(device)
                    token_type_ids = batch['token_type_ids'].to(device)
                    attention_mask = batch['attention_mask'].to(device)
                    labels = batch['label'].to(device)
                    outputs = model_engine(input_ids=input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask, labels=labels)
                    loss = outputs.loss
                    val_loss += loss.data / val_steps
                    val_loss = hvd.allreduce(val_loss, op=hvd.Sum)
                    val_loss = val_loss / world_size
            ######

            # here we report the training and validation losses, and the elapsed time per epoch
            # we also implement a basic early stopping srategy
            ######
            if rank == 0:
                log_to_driver('\nTrain loss after itaration %i: %f' % (n+1, train_loss))
                log_to_driver('Validation loss after itaration %i: %f' % (n+1, val_loss))
                end_time = time.time()
                epoch_mins, epoch_secs = epoch_time(start_time, end_time)
                log_to_driver('Time: %im %is' % (epoch_mins, epoch_secs))
                
                if best_loss == None:
                    best_loss = val_loss
                elif best_loss - val_loss > early_stop_min_delta:
                    best_loss = val_loss
                    early_stop_counter = 0
                elif best_loss - val_loss < early_stop_min_delta:
                    early_stop_counter += 1
                    log_to_driver(f'Early stopping counter {early_stop_counter} of {early_stop_patience}')
                    if early_stop_counter >= early_stop_patience:
                        log_to_driver('Early stopping')
                        flag_tensor = torch.ones(1).to(device)

            train_losses.append(train_loss.cpu())
            val_losses.append(val_loss.cpu())

            flag_tensor = hvd.allreduce(flag_tensor, op=hvd.Sum)
            if flag_tensor == 1:
                break
    ######

    # at the end of training, we save the trained model files and tracked losses
    ######
    if rank == 0:
        log_to_driver('Saving best model to %s' % model_folder)
        model_to_save = model.module if hasattr(model, 'module') else model
        model_to_save.save_pretrained(model_folder)
        with open(model_folder + '/train_losses.pkl', 'wb') as f:
            pickle.dump(train_losses, f)
        with open(model_folder + '/val_losses.pkl', 'wb') as f:
            pickle.dump(val_losses, f)
    ######

We use the [*HorovodRunner*](https://docs.microsoft.com/en-us/azure/databricks/applications/machine-learning/train-model/distributed-training/horovod-runner) class to submit the Spark job that runs the model training function with *Horovod*.

In [0]:
hr = HorovodRunner(np=num_workers)
hr.run(finetune_model)

HorovodRunner will only stream logs generated by :func:`sparkdl.horovod.log_to_driver` or
:class:`sparkdl.horovod.tensorflow.keras.LogCallback` to notebook cell output. If want to stream all
logs to driver for debugging, you can set driver_log_verbosity to 'all', like `HorovodRunner(np=2,
driver_log_verbosity='all')`.
The global names read or written to by the pickled function are {'hvd': None, 'os': None, 'log_to_driver': None, 'torch': None, 'random': None, 'np': None, 'AutoConfig': None, 'AutoModelForSequenceClassification': None, 'AdamW': None, 'len': None, 'converter_train': None, 'get_linear_schedule_with_warmup': None, 'deepspeed': None, 'converter_val': None, 'iter': None, 'max': None, 'range': None, 'time': None, 'next': None, 'hasattr': None, 'open': None, 'pickle': None, 'any': None, 'int': None}.
The pickled object size is 10207 bytes.

### How to enable Horovod Timeline? ###
HorovodRunner has the ability to record the timeline of its activity with Horovod  Timeline. To
rec

Delete the *Petastorm* cache files defined earlier.

In [0]:
converter_train.delete()
converter_val.delete()